In [0]:
import random
import numpy as np
import time

In [2]:
from google.colab import files
uploaded = files.upload()

Saving move.npy to move.npy


In [0]:
singleScore=[0,0,4,16,48,128,320,768,1792,4096,9216,20480,45056,98304,212992,458752,983040]
moveDict=np.load('move.npy')

def add_two(mat):
    indexs=np.argwhere(mat==0)
    index=np.random.randint(0,len(indexs))
    mat[tuple(indexs[index])] = 1
    return mat

def game_state(mat):
    return 'not over' if np.any(mat == 0) or np.any(mat[:, 0:-1]==mat[:, 1:]) or np.any(mat[0:-1, :]==mat[1:, :]) else 'lose'

def move(list):
    return moveDict[list[0],list[1],list[2],list[3],:]

def lookup(x):
    return singleScore[x]

lookup = np.vectorize(lookup)

def getScore(matrix):
    return np.sum(lookup(matrix))

def isSame(grid1,grid2):
    return np.all(grid1==grid2)

def moveGrid(grid,i):
    # new=np.zeros((4,4),dtype=np.int)
    new = None
    if i==0:
        # move up
        grid=np.transpose(grid)
        new = np.stack([move(grid[row,:]) for row in range(4)], axis = 0).astype(int).T
    elif i==1:
        # move left
        new = np.stack([move(grid[row,:]) for row in range(4)], axis = 0).astype(int)
    elif i==2:
        # move down
        grid=np.transpose(grid)
        new = np.stack([np.flip(move(np.flip(grid[row,:]))) for row in range(4)], axis = 0).astype(int).T
    elif i==3:
        # move right
        new = np.stack([np.flip(move(np.flip(grid[row,:]))) for row in range(4)], axis = 0).astype(int)
    return new

In [0]:
def getMove(grid, numBackgroundRuns):
    ls = []
    for i in range(4):
        nextGrid = moveGrid(grid, i)
        score = 0
        if not isSame(grid, nextGrid):
            score = randomPolicyUntilGameOver(nextGrid, numBackgroundRuns)
        ls.append((score, i, nextGrid))
        
    return random.choice([grid for score, _, grid in ls if score == max(ls)[0]])

def randomPolicyUntilGameOver(grid, numBackgroundRuns):
    score = 0
    for i in range(numBackgroundRuns):
        curGrid = grid.copy()
        while True:
            curGrid = add_two(curGrid)
            board_list = randomMove(curGrid)
            if not board_list:
                break
            curGrid = random.choice(board_list)
        score += getScore(curGrid)
    return score

def randomMove(grid):
    board_list = []
    for i in range(4):
        newGrid=moveGrid(grid, i)
        if not isSame(grid,newGrid):
            board_list.append(newGrid)
    return board_list

In [6]:
ITERATION=100
NUM_BACKGROUND_RUNS=10
stat=[]

def main():
    for iteration in range(ITERATION):
        start=time.time()
        step=0
        matrix=np.zeros((4,4),dtype=np.int)
        matrix = add_two(matrix)
        while True:
            matrix = add_two(matrix)
            if game_state(matrix)=='lose':
                break

            matrix=getMove(matrix, NUM_BACKGROUND_RUNS)
            step+=1

        print("Iteration %d, Step %d, max %d, Score %d, \r\n" % (iteration, step, 2**np.max(matrix), getScore(matrix)))

if __name__ == '__main__':
    main()

Iteration 0, Step 1063, max 1024, Score 16732, 

Iteration 1, Step 1941, max 2048, Score 35192, 

Iteration 2, Step 1060, max 1024, Score 16680, 

Iteration 3, Step 1054, max 1024, Score 16588, 

Iteration 4, Step 1072, max 1024, Score 16792, 

Iteration 5, Step 1052, max 1024, Score 16576, 

Iteration 6, Step 2014, max 2048, Score 36052, 

Iteration 7, Step 283, max 256, Score 3236, 

Iteration 8, Step 2034, max 2048, Score 36636, 

Iteration 9, Step 1065, max 1024, Score 16744, 

Iteration 10, Step 1348, max 2048, Score 24120, 

Iteration 11, Step 1018, max 1024, Score 16108, 

Iteration 12, Step 810, max 1024, Score 12588, 

Iteration 13, Step 1056, max 1024, Score 16600, 

Iteration 14, Step 1953, max 2048, Score 35352, 

Iteration 15, Step 1052, max 1024, Score 16552, 

Iteration 16, Step 1041, max 1024, Score 16472, 

Iteration 17, Step 804, max 1024, Score 12544, 

Iteration 18, Step 1007, max 1024, Score 16012, 

Iteration 19, Step 1574, max 2048, Score 27908, 

Iteration 20, S